In [66]:
import os, sys, string, time, re
import requests, json, urllib, urllib2, base64
import pymongo
import pandas as pd
from sklearn.metrics import accuracy_score
from multiprocessing import Pool, Lock, Queue, Manager

In [11]:
df_100 = pd.read_csv("df_100.csv")
tweetcollection = list(df_100['text'])

In [16]:
#need a list of dictionaries like [{text:aodijgaodsigma},{text:oaidmgoaimdoa}]
adict = []
for i in tweetcollection:
    adict.append({'text':i})

In [19]:
print adict[0]['text']

RT @Cubs: .@kschwarb12 goes dead-center! Back-to-back blasts give the #Cubs a 4-1 lead in the 5th! #LetsGo


In [38]:
#lets bring in the preprocessed file too
df_100_pp = pd.read_csv("df_100_pp.csv")
tweetcollection2 = list(df_100_pp['text'])
adict2 = []
for i in tweetcollection2:
    adict2.append({'text':i})

In [70]:
def main():

    # Establish credentials for Twitter and AlchemyAPI
    credentials = get_credentials()

    # Pull Tweets down from the Twitter API
    raw_tweets = adict
    
    # Enrich the body of the Tweets using AlchemyAPI
    enriched_tweets = enrich(credentials, raw_tweets)

    # Store sentiment-analyzed data in MongoDB
    store(enriched_tweets)
    
    # Print some interesting results to the screen
    print_results()

    return

In [71]:
def get_credentials():
    creds = {}
    creds['apikey'] = '14c53dfe439a2e3ff87e7750269a79630da88746'
    return creds

In [72]:
def enrich(credentials, tweets, sentiment_target = ''):
    # Prepare to make multiple asynchronous calls to AlchemyAPI
    apikey = credentials['apikey']
    pool = Pool(processes = 10)
    mgr = Manager()
    result_queue = mgr.Queue()
    # Send each Tweet to the get_text_sentiment function
    for tweet in tweets:
        #what does sentiment_target do here? look up get_text_sentiment()
        pool.apply_async(get_text_sentiment, (apikey, tweet, sentiment_target, result_queue))

    pool.close()
    pool.join()
    collection = []
    while not result_queue.empty():
        collection += [result_queue.get()]
    
    print "Enrichment complete! Enriched %d Tweets" % len(collection)
    return collection

In [73]:
def get_text_sentiment(apikey, tweet, target, output):

    # Base AlchemyAPI URL for targeted sentiment call
    alchemy_url = "http://access.alchemyapi.com/calls/text/TextGetTextSentiment"
    
    # Parameter list, containing the data to be enriched
    parameters = {
        "apikey" : apikey,
        "text"   : tweet['text'],
        "outputMode" : "json",
        "showSourceText" : 1
        }

    try:

        results = requests.get(url=alchemy_url, params=urllib.urlencode(parameters))
        response = results.json()

    except Exception as e:
        print "Error:", e
        return

    try:
#         if 'OK' != response['status'] or 'docSentiment' not in response:
#             print "Problem finding 'docSentiment' in HTTP response from AlchemyAPI"
#             print response
#             print "HTTP Status:", results.status_code, results.reason
#             print "--"
#             return

        tweet['sentiment'] = response['docSentiment']['type']
        tweet['score'] = 0.
        if tweet['sentiment'] in ('positive', 'negative'):
            tweet['score'] = float(response['docSentiment']['score'])
        output.put(tweet)

    except Exception as e:
        tweet['sentiment'] = 'neutral'
        tweet['score'] = 0
        output.put(tweet)
        print "Error:", e
        print "Request:", results.url
        print "Response:", response

    return

In [74]:
def store(tweets):
    # Instantiate your MongoDB client
    mongo_client = pymongo.MongoClient()
    # Retrieve (or create, if it doesn't exist) the twitter_db database from Mongo
    db = mongo_client.Alchemy
   
    db_tweets = db.ResultsRaw

    for tweet in tweets:
        db_id = db_tweets.insert(tweet)
 
    db_count = db_tweets.count()
    
    print "# of Sentiment-Analyzed Tweets stored in MongoDB: %d" % db_count

    return

In [75]:
def print_results():

    print ''
    print ''
    print '###############'
    print '#    Stats    #'
    print '###############'
    print ''
    print ''    
    
    db = pymongo.MongoClient().Alchemy
    tweets = db.ResultsRaw

    num_positive_tweets = tweets.find({"sentiment" : "positive"}).count()
    num_negative_tweets = tweets.find({"sentiment" : "negative"}).count()
    num_neutral_tweets = tweets.find({"sentiment" : "neutral"}).count()
    num_tweets = tweets.find().count()

    if num_tweets != sum((num_positive_tweets, num_negative_tweets, num_neutral_tweets)):
        print "Counting problem!"
        print "Number of tweets (%d) doesn't add up (%d, %d, %d)" % (num_tweets, 
                                                                     num_positive_tweets, 
                                                                     num_negative_tweets, 
                                                                     num_neutral_tweets)
        sys.exit()

    most_positive_tweet = tweets.find_one({"sentiment" : "positive"}, sort=[("score", -1)])
    most_negative_tweet = tweets.find_one({"sentiment" : "negative"}, sort=[("score", 1)])

    mean_results = tweets.aggregate([{"$group" : {"_id": "$sentiment", "avgScore" : { "$avg" : "$score"}}}])

#     avg_pos_score = mean_results['result'][2]['avgScore'] 
#     avg_neg_score = mean_results['result'][1]['avgScore'] 
    
    print "SENTIMENT BREAKDOWN"
    print "Number (%%) of positive tweets: %d (%.2f%%)" % (num_positive_tweets, 100*float(num_positive_tweets) / num_tweets)
    print "Number (%%) of negative tweets: %d (%.2f%%)" % (num_negative_tweets, 100*float(num_negative_tweets) / num_tweets)
    print "Number (%%) of neutral tweets: %d (%.2f%%)" % (num_neutral_tweets, 100*float(num_neutral_tweets) / num_tweets)
    print ""

#     print "AVERAGE POSITIVE TWEET SCORE: %f" % float(avg_pos_score)
#     print "AVERAGE NEGATIVE TWEET SCORE: %f" % float(avg_neg_score)
#     print ""

    print "MOST POSITIVE TWEET"
    print "Text: %s" % most_positive_tweet['text']
#     print "User: %s" % most_positive_tweet['screen_name']
#     print "Time: %s" % most_positive_tweet['time']
    print "Score: %f" % float(most_positive_tweet['score'])
    print ""

    print "MOST NEGATIVE TWEET"
    print "Text: %s" % most_negative_tweet['text']
#     print "User: %s" % most_negative_tweet['screen_name']
#     print "Time: %s" % most_negative_tweet['time']
    print "Score: %f" % float(most_negative_tweet['score'])
    return


In [76]:
main()

Enrichment complete! Enriched 100 Tweets
# of Sentiment-Analyzed Tweets stored in MongoDB: 100Error: 'docSentiment'
Error: 'docSentiment'
Error: 'docSentiment'
Error: 'docSentiment'
Error: 'docSentiment'
Error: 'docSentiment'
Request: http://access.alchemyapi.com/calls/text/TextGetTextSentiment?text=RT+%40MLB%3A+%23FireEmoji++https%3A%2F%2Ft.co%2FZN7FshqMkp&apikey=14c53dfe439a2e3ff87e7750269a79630da88746&outputMode=json&showSourceText=1
Request: http://access.alchemyapi.com/calls/text/TextGetTextSentiment?text=%23cubs+http%3A%2F%2Ft.co%2FQOtoWxPTRi&apikey=14c53dfe439a2e3ff87e7750269a79630da88746&outputMode=json&showSourceText=1
Request: http://access.alchemyapi.com/calls/text/TextGetTextSentiment?text=%40Cubs+%F0%9F%94%A5%F0%9F%94%A5%F0%9F%94%A5%F0%9F%94%A5%F0%9F%91%8C%F0%9F%91%8C%F0%9F%91%8C%F0%9F%91%8C%F0%9F%91%8D&apikey=14c53dfe439a2e3ff87e7750269a79630da88746&outputMode=json&showSourceText=1
Request: http://access.alchemyapi.com/calls/text/TextGetTextSentiment?text=RT+%405kl%3A+%40

In [50]:
label_list = [1,1,1,1,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,-1,1,0,1,0,0,1,1,0,-1,1,0,0,-1,0,0,0,0,1,0,1,1,0,1,1,1,0,1,0,1,0,0,
              1,1,1,1,0,1,1,0,1,0,0,1,1,1,1,1,0,1,1,0,1,0,1,1,0,1,-1,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,1,0,1,1,1,0,0]

In [64]:
#bring in the processed alchemy scores from MongoDB to Pandas

alchemyscores = pymongo.MongoClient().Alchemy.Results
cursor = alchemyscores.find()
cubsDF = pd.DataFrame(list(cursor))    

In [65]:
cubsDF['pol_class'] = cubsDF['sentiment']
cubsDF['pol_class'] = cubsDF['pol_class'].replace("neutral",0).replace("positive",1).replace("negative",-1)
print cubsDF

                         _id     score sentiment  \
0   55cea7fcb187a81013554edb  0.000000   neutral   
1   55cea7fcb187a81013554edc  0.670829  positive   
2   55cea7fcb187a81013554edd -0.261398  negative   
3   55cea7fcb187a81013554ede  0.839020  positive   
4   55cea7fcb187a81013554edf -0.315540  negative   
5   55cea7fcb187a81013554ee0  0.261258  positive   
6   55cea7fcb187a81013554ee1  0.000000   neutral   
7   55cea7fcb187a81013554ee2  0.000000   neutral   
8   55cea7fcb187a81013554ee3  0.000000   neutral   
9   55cea7fcb187a81013554ee4  0.000000   neutral   
10  55cea7fcb187a81013554ee5  0.000000   neutral   
11  55cea7fcb187a81013554ee6  0.000000   neutral   
12  55cea7fcb187a81013554ee7  0.261258  positive   
13  55cea7fcb187a81013554ee8 -0.225455  negative   
14  55cea7fcb187a81013554ee9 -0.341243  negative   
15  55cea7fcb187a81013554eea -0.397927  negative   
16  55cea7fcb187a81013554eeb  0.000000   neutral   
17  55cea7fcb187a81013554eec  0.210043  positive   
18  55cea7fc

In [67]:
#accuracy for Alchemy with Pre-processed tweets
accuracy_score(label_list, list(cubsDF.pol_class))
#fucking hell... 36% accuracy. yay.

0.35999999999999999

In [68]:
cubsDF.pol_class.value_counts()

 0    53
 1    30
-1    17
dtype: int64

In [69]:
print dict((i, label_list.count(i)) for i in label_list)

{0: 41, 1: 55, -1: 4}


In [77]:
#bring in the non-processed alchemy scores from MongoDB to Pandas

alchemyscores2 = pymongo.MongoClient().Alchemy.ResultsRaw
cursor2 = alchemyscores2.find()
cubsDF2 = pd.DataFrame(list(cursor2))  

In [78]:
cubsDF2['pol_class'] = cubsDF2['sentiment']
cubsDF2['pol_class'] = cubsDF2['pol_class'].replace("neutral",0).replace("positive",1).replace("negative",-1)
accuracy_score(label_list, list(cubsDF2.pol_class))
#accuracy for Alchemy with non-processed tweets is 47%

0.46999999999999997